In [1]:
#!pip install pandas_datareader
#!pip install plotly
#!pip install chart-studio
#!pip install tensorflow
#!pip install sklearn
#!pip install tensorflow

In [2]:
import datetime as dt
import numpy as np
import pandas as pd


from numpy import log as ln
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader as web
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

#import tensorflow as tf
from statsmodels.tools.eval_measures import rmse
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, LeakyReLU
from sklearn.metrics import r2_score, mean_squared_error

from keras.preprocessing.sequence import TimeseriesGenerator

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


The intent with this project is to use an LSTM-model to predict the price based on a timesseries of stock prices and comparing the results between features from [Predicting the Direction of Stock Market Index](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4873195/#pone.0155133.s001). There is also a comparison between the LSTM model and linear regression.

## Functions

In [3]:
def reshape_x(dataframe, time_string):
    """Converts a timeseries dataframe into the given time_string format"""
    df = dataframe.resample(time_string, on="DateTime").agg({
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Close": "last",
        "Volume": "sum"
    })
    return df.dropna()

In [4]:
def hit_ratio(dataframe, col1, col2):
    """Return the hit ratio between predicted values and the actual values in a dataframe.
    hit_ratio(dataframe, col1, col2)
    col1 and col2 are the column names that are to be compared.
    """
    dataframe["Hit Ratio"] = np.where(dataframe[col1].shift(1) - dataframe[col1] / (dataframe[col2].shift(1) - dataframe[col2]) > 0, 1, 0)
    return dataframe["Hit Ratio"].mean() * 100

In [5]:
def scores(y_test, df):
    """Calculates the scores pased on the prediction and the expected values"""
    r2, hit, mse = r2_score(y_test, df["Predicted"] ), hit_ratio(df, "Actual test", "Predicted" ), mean_squared_error(y_test, df["Predicted"])
    print("R2 Prediction Score: ", r2)
    print("Hit ratio: ", hit)
    print("Mean square error: ", mse)
    score = [r2, hit, mse]
    #return scores[0],scores[1]
    return score

In [6]:
def calculate_features(dataframe, frequency):
    """This function adds features to a dataframe.
    
    This function adds features to a dataframe. It will 
    Ln: Lowest price in the last n days
    Hn: Highest price in the last n days
    RoC: % Change from the current t value, compared to t - n 
    S%D: Stochastic oscillator %D
    S%K: Stochastic oscillat %K
    SYt: Return of the index at time t
    ASY5: Average return in the last n days
    """
    dataframe["RoC"] = ((dataframe["Close"] / dataframe["Close"].shift(frequency)) - 1) * 100
    dataframe["Ln"] = dataframe["Low"].rolling(window=frequency).min()
    dataframe["Hn"] = dataframe["High"].rolling(window=frequency).max()
    dataframe["S%K"] = (dataframe["Close"] - dataframe["Ln"]) / (dataframe["Hn"] - dataframe["Ln"]) * 100
    dataframe["S%D"] = dataframe["S%K"].rolling(window=frequency).mean()
    dataframe["SYt"] = (ln(dataframe["Close"]) - ln(dataframe["Close"].shift(1))) * 100
    dataframe["ASY5"] = dataframe["SYt"].rolling(5).mean()
    dataframe["MA5"] = dataframe["Close"].rolling(5).mean()
    dataframe["A"] = np.where(dataframe["Close"] > dataframe["Close"].shift(1), 1, 0)
    dataframe["PSY12"] = dataframe["A"].rolling(frequency).sum() * 2
    return dataframe.dropna()

In [7]:
def draw_graph(dataframe, columns, colors, title):
    """
    Draws a pyplot graph with up to 3 lines depending on the input.
    """
        
    layout = go.Layout(
        yaxis2 = dict(
            title="Normalized value", 
            side="right",
            tickmode = "auto"
        ),
        title = title)
    fig = go.Figure(layout = layout)
    
    for column, color in zip(columns, colors):
        fig.add_trace(
            add_line(dataframe.index, dataframe[column], color, column)
        )
    
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=7,
                        label="7d",
                        step="day",
                        stepmode="backward"),
                    dict(count=1,
                         label="1m",
                         step="month",
                         stepmode="backward"),
                    dict(count=6,
                         label="6m",
                         step="month",
                         stepmode="backward"),
                    dict(count=1,
                         label="YTD",
                         step="year",
                         stepmode="todate"),
                    dict(count=1,
                         label="1y",
                         step="year",
                         stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    fig.show()
    
def add_line(x_data,y_data, color, title):
    """Adds a pyplot line to a predefine pyplot figure."""
    return go.Scatter(
        mode="lines",
        x=x_data,
        y=y_data,
        line = dict(width=1),
        marker = dict(color=color),
        yaxis = "y1", name = title
    )

In [8]:
def train_test_split(dataframe, x_col, y_col, forecast_days):
    """Convert a dataframe into x_train, x_test, y_train, y_test based on the given columns and forecast day"""
    x = np.array(dataframe[x_col][:-forecast_days])
    y = np.array(dataframe[y_col][:-forecast_days])
    
    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tscv.split(x):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
    return x_train, x_test, y_train, y_test

In [9]:
def linear_regression(settings):
    """Settings contains a dataframe, the columns and the forecast day coun.
    The function create a linear regression model that makes a prediction based on the given data and forecast."""
    dataframe = settings["Data"]
    x_train, x_test, y_train, y_test = train_test_split(dataframe, settings["Cols"], "Label", settings["Forecast"])
    
    #scaler = MinMaxScaler()
    #X = scaler.fit_transform(x_train, y_train)
    #x_test = scaler.transform(x_test)
    #y_test = scaler.transform(y_test.reshape(-1, 1))
    #x_train, y_train = X[0], X[1],
    
    

    lr = LinearRegression()
    lr_model = lr.fit(x_train, y_train)
    
    lr_prediction = lr.predict(x_test)
    
    #lr_prediction = scaler.inverse_transform(lr_prediction)
    #y_test = scaler.inverse_transform(y_test)
    
    data = {"Predicted": lr_prediction, "Actual test": y_test}
    
    lr_df = pd.DataFrame(index=dataframe[-len(y_test):].index, data=data)
    
    #print("Test score:", r2_score(y_test, x_test))
    score = scores(y_test, lr_df)

    return lr_df, score

In [10]:
def lstm_model(settings, features=3):
    """Creates an LSTM model that makes a prediction based on the dates of the test data. 
    Returns the predicted data and the calculated score of the model"""
    dataframe = settings["Data"]
    x_train, x_test, y_train, y_test = train_test_split(dataframe, settings["Cols"], "Label", settings["Forecast"])
    n_input = 1
    n_features = features
    generator = TimeseriesGenerator(x_train, y_train, length=n_input, batch_size=6)
    #generator_test = TimeseriesGenerator(x_test, y_test, length=n_input, batch_size=6)
    
    model = Sequential()
    model.add(LSTM(200, return_sequences=True, activation='relu', input_shape=(n_input, n_features)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dropout(0.4))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    history = model.fit_generator(generator, epochs=20)
    
    x_test_reshape = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
    
    lstm_prediction = model.predict(x_test_reshape)
    #lstm_prediction = model.predict(generator_test)
    reshaped_prediction = lstm_prediction.reshape(lstm_prediction.shape[0])
    data = {"Predicted": reshaped_prediction, "Actual test": y_test}
    prediction_data = pd.DataFrame(index=dataframe[-len(y_test):].index, data=data)
    
    score = scores(y_test, prediction_data)
    
    return prediction_data, score

## implementation

My chosen n is 12, which essentially means 14 days with this dataset. This dataset does not include saturdays so I decided to account for that. I base this on [an investopedia article](https://www.investopedia.com/terms/s/stochasticoscillator.asp) that states 14 days is somewhat standard.

The features used in this project are taken from [Predicting the Direction of Stock Market Index](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4873195/#pone.0155133.s001). They are split into Type 1 and Type 2 features, and I will be using two of each.

The type 1 features I have chosen are

#### Rate of change
C<sub>t</sub> / C<sub>(t-n)</sub> * 100

#### Stochastic %D
$\sum_{i=0}^{n-1}$ %K<sub>(t-i)</sub> / n || Moving average of the last n days of the %K

%K: (C<sub>t</sub> - L<sub>n</sub>) / (H<sub>n</sub> - L<sub>n</sub>) * 100

The type 2 features I have chosen are

#### ASY5 (Average return in the last 5 days)

$\sum_{i=0}^{5}$ SY<sub>t-i</sub>

SY<sub>t</sub> = (ln(C<sub>t</sub>) - ln(C<sub>t-1</sub>)) * 100

#### PSY12 (Ratio of rising periods over a 12 day period

(A/12) * 100%

***See the "calculate_features" function for how thease features are implemented coded***

### Other variables

* C<sub>t</sub>: Closing price at time t
* L<sub>t</sub>: Lowest price at time t
* H<sub>t</sub>: Highest price at time t


* L<sub>n</sub>: Lowest price in the last n days
* H<sub>n</sub>: Highest price in the last n days


* MA<sub>n</sub>: Moving average of the price value in the last n days
* Up<sub>t</sub>: Upward price change at the time t
* Dw<sub>t</sub>: Downward price change at the time t
* A: Shows rising days and days that do not rise

## Creating our data

* Data does not include saturdays
* I'm basing my data on a 2 weeks (14 day window), since saturdays aren't included in the data I'm choosing to go with n = 12
* I create 3 dataframes since i want to compare 3 sets of features

In [11]:
df_orig = pd.read_csv("EURUSD1m.csv", parse_dates={"DateTime": ["Date","Timestamp"]})
df = df_orig

In [12]:
#df_minutes = df
#df_minutes = df_minutes.set_index("DateTime")
#df_hours = reshape_x(df, "H")
#df_minutes = calculate_features(df, n * 24 * 60)
#df_hours = calculate_features(df_hours, n * 24)

df_daily = reshape_x(df, "d")
n = 14 - 2 # 2 weeks - missing saturdays
df_daily = calculate_features(df_daily, n)

scaler = MinMaxScaler()
s_cols = ["Close", "RoC", "S%D", "PSY12", "ASY5"]
scaled = scaler.fit_transform(df_daily[s_cols])
scaled_daily_df = pd.DataFrame(data=scaled, index=df_daily.index, columns=s_cols)

col_t1 = ["Close", "RoC", "S%D"]
col_t2 = ["Close", "PSY12", "ASY5"]
col_comb = ["Close", "RoC", "S%D", "PSY12", "ASY5"]
df_t1 = scaled_daily_df[col_t1] # Tier 1 features dataframe
df_t2 = scaled_daily_df[col_t2] # Tier 2 features dataframe
df_comb = scaled_daily_df[col_comb] # Tier 1 & 2 features dataframe
#df_t1 = df_daily[col_t1] # Tier 1 features dataframe
#df_t2 = df_daily[col_t2] # Tier 2 features dataframe
#df_comb = df_daily[col_comb] # Tier 1 & 2 features dataframe

### Add our prediction to each dataframe


In [13]:
forecast_days = 1
df_t1["Label"] = df_t1["Close"].shift(forecast_days)
df_t2["Label"] = df_t2["Close"].shift(forecast_days)
df_comb["Label"] = df_comb["Close"].shift(forecast_days)
df_t1.dropna(inplace=True)
df_t2.dropna(inplace=True)
df_comb.dropna(inplace=True)

### Create settings for more practical access to the dataframes

In [14]:
settings_t1 = {
    "Data": df_t1,
    "Cols": col_t1,
    "Forecast": forecast_days
}


settings_t2 = {
    "Data": df_t2,
    "Cols": col_t2,
    "Forecast": forecast_days
}

settings_comb = {
    "Data": df_comb,
    "Cols": col_comb,
    "Forecast": forecast_days
}

## Visualization of the T1 features

In [15]:
draw_graph(df_t1, col_t1, ["#32a852","#eb1717","#0059ff"], "Data visualized")

There isn't too much to say about this data in this format. The close price doesn't fluctuate too much since that day to day value is based on an actual price. What it does do is fluctuate quite a bit over the span of 10 years. The RoC and S%D are both some ratio which means it also fluctuates heavily. The values can jump between 0 and 1 in a couple days time.

# Linear regression

## Linear regression with type 1 features

In [16]:
lr_df_t1, lr_scores_t1 = linear_regression(settings_t1)

R2 Prediction Score:  0.975512894300977
Hit ratio:  52.9980657640232
Mean square error:  6.760934917689821e-05


## Linear regression with type 2 features

In [17]:
lr_df_t2, lr_scores_t2 = linear_regression(settings_t2)

R2 Prediction Score:  0.9772550900808983
Hit ratio:  51.45067698259188
Mean square error:  6.2799114587846e-05


## Linear regression with type 1&2 features

In [18]:
lr_df_comb, lr_scores_comb = linear_regression(settings_comb)

R2 Prediction Score:  0.9776736534356402
Hit ratio:  51.25725338491296
Mean square error:  6.164345346761303e-05


The linear regression model works pretty well. It is extremely light-weight and runs quickly which means results like this are surprisingly good.

The hit ratios are surprisingly good, however I'm sceptical that the type 1 model would perform as well on real data. 
It does surprise me a little that the combined features performs slightly worse than only the type 2 features.

Overall the R2 score of the prediction is reasonable for all the different features, and although I'm not showing it now the R2 scores for the test data were implying that there wasn't too much overfitting.

In [19]:
draw_graph(lr_df_t1, ["Predicted", "Actual test"], ["#eb1717","#0059ff"], "Linear regression with type 1 features, predicted vs actual")

In [20]:
draw_graph(lr_df_t2, ["Predicted", "Actual test"], ["#eb1717","#0059ff"], "Linear regression with type 2 features, predicted vs actual")

In [21]:
draw_graph(lr_df_comb, ["Predicted", "Actual test"], ["#eb1717","#0059ff"], "Linear regression with type 1&2 features, predicted vs actual")

# LSTM model
## LSTM model with type 1 features

In [22]:
lstm_t1_prediction, lstm_t1_scores = lstm_model(settings_t1)

Epoch 1/20
431/431 [==============================] - 2s 5ms/step - loss: 0.0320
Epoch 2/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0043
Epoch 3/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0039
Epoch 4/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0034
Epoch 5/20
431/431 [==============================] - 2s 3ms/step - loss: 0.0027
Epoch 6/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0027
Epoch 7/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0024
Epoch 8/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0019
Epoch 9/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0019
Epoch 10/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0015
Epoch 11/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0016
Epoch 12/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0014
Epoch 13/20
431/431 [====

## LSTM model with type 2 features

In [23]:
lstm_t2_prediction, lstm_t2_scores = lstm_model(settings_t2)

Epoch 1/20
431/431 [==============================] - 2s 5ms/step - loss: 0.0302
Epoch 2/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0048
Epoch 3/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0037
Epoch 4/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0032
Epoch 5/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0029
Epoch 6/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0025
Epoch 7/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0023
Epoch 8/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0020
Epoch 9/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0017
Epoch 10/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0015
Epoch 11/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0014
Epoch 12/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0013
Epoch 13/20
431/431 [====

## LSTM model with type 1&2 features

In [24]:
lstm_comb_prediction, lstm_comb_scores = lstm_model(settings_comb, features=5)

Epoch 1/20
431/431 [==============================] - 2s 5ms/step - loss: 0.0365
Epoch 2/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0046
Epoch 3/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0039
Epoch 4/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0032
Epoch 5/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0030
Epoch 6/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0026
Epoch 7/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0026
Epoch 8/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0019
Epoch 9/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0019
Epoch 10/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0016
Epoch 11/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0015
Epoch 12/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0013
Epoch 13/20
431/431 [====

## How does the LSTM model perform

Depending on the random seed, each version of the model can perform slightly better. Overall the model with combined features has performed better. 

The type 1 features model fluctuates around 52% hitrate, while the combined one is in the upper 52% with the type model somewhere in between the two. 

The R2 score sits drops to rouglhy 90% for the combined model, and the type 1 model and type 2 model both sit at roughly 95%. These are reasonable scores. Occasionally however, the R2 score for this LSTM model can suddenly drop very sharply, and I've seen an R2 score for the  type 1 model at 0.5. I'm not sure why this happens.

In [25]:
draw_graph(lstm_t1_prediction, ["Predicted", "Actual test"], ["#eb1717","#0059ff"], "LSTM model with type 1 features, predicted vs actual")

In [26]:
draw_graph(lstm_t2_prediction, ["Predicted", "Actual test"], ["#eb1717","#0059ff"], "LSTM model with type 2 features, predicted vs actual")

In [27]:
draw_graph(lstm_comb_prediction, ["Predicted", "Actual test"], ["#eb1717","#0059ff"], "LSTM model with type 1&2 features, predicted vs actual")

## Hit ratio table

In [28]:
stat_table = pd.DataFrame(
    np.array([lr_scores_t1, lr_scores_t2, lr_scores_comb, lstm_t1_scores, lstm_t2_scores, lstm_comb_scores]),
    columns = ["R2", "Hit", "MSE"],
    index = ["LR T1", "LR T2", "LR Comb", "LSTM T1", "LSTM T2", "LSTM Comb"]
)

In [29]:
stat_table

,R2,Hit,MSE
LR T1,0.975513,52.998066,0.000068
LR T2,0.977255,51.450677,0.000063
LR Comb,0.977674,51.257253,0.000062
LSTM T1,0.952027,52.611219,0.000132
LSTM T2,0.972268,52.804642,0.000077
LSTM Comb,0.785637,53.191489,0.000592


This table will shift between each restart of the notebook, however overall the hit ratio for the LSTM model is better.

The linear regression models hit ratio is overall about 1.5% worse than the lSTM model which is a larger gap than I personally expected. 

## Comparison of the models 

The linear model is quick and efficient while still giving good results. The LSTM model on the other hand is quite a bit slower, but it does have a signicant increase in the results.

When it came to the different features, the type 2 features by themselves weren't impressive. Combining them with the type 1 features then gave quite a decent increase in performance overall I feel. 

I tried using the Moving average type 2 feature alongside the ASY5 for the LSTM model early on, however that gave me extremely poor results. I even ended up with a 49% hit ratio once.

I would personally definitely pick the LSTM model for any kind of extended use, and making sure to retrain it often enough. 

## Investment rule

My investment rule is a simple sell high, buy low. Since our predictions are made 1 day ahead, I decided to buy and sell on a daily basis as well. The risk would increase significantly if I made buy and sell decisions based on my predictions since my hit rate is only slightly above 50%.

In [41]:
investing = pd.DataFrame(lstm_comb_prediction["Predicted"])

In [42]:
col = "Predicted"
conditions = [investing[col] > investing[col].shift(-1), investing[col] < investing[col].shift(-1), investing[col] == investing[col].shift(-1)]
choices = ["Sell", "Buy", "Hold"]
investing["Invest"] = np.select(conditions, choices, default=np.nan)

In [43]:
investing.head(10)

,Predicted,Invest
DateTime,,
2018-05-08,0.364693,Sell
2018-05-09,0.350673,Sell
2018-05-10,0.345640,Buy
2018-05-11,0.360510,Buy
2018-05-13,0.366431,Buy
2018-05-14,0.369277,Sell
2018-05-15,0.365090,Sell
2018-05-16,0.340147,Sell
2018-05-17,0.338278,Sell


## Improvements that could be made

Scaling of the data (Normalization) to be done on the train data instead of all the data. 

Store all the output (hit ratio, R2 score etc.) in a CSV file separate for each model, to be able to accurately describe how each model performs based on statistics. Alternatively this could be done by removing the random seeds in the notebook however that isn't necessarily a good final result.